<a href="https://colab.research.google.com/github/WIVIV/financial_analysis/blob/master/Percent_Change_V1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install simfin

     |████████████████████████████████| 61kB 3.1MB/s 


In [0]:
import pandas_datareader as pdr

In [0]:
#imports
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.optimize import differential_evolution
from datetime import datetime, timedelta
import os

# Import the main functionality from the SimFin Python API.
import simfin as sf

# Import names used for easy access to SimFin's data-columns.
from simfin.names import *

In [7]:
# Version of the SimFin Python API.
sf.__version__

'0.5.0'

In [0]:
# SimFin data-directory.
sf.set_data_dir('~/simfin_data/')
# SimFin load API key or use free data.
sf.load_api_key(path='~/simfin_api_key.txt', default_key='free')
# Seaborn set plotting style.
sns.set_style("whitegrid")

In [14]:
%%time
# Data for USA.
market='us'

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 6.44 µs


In [78]:
df_prices = sf.load_shareprices(variant='daily', market=market)
df_prices_latest = sf.load_shareprices(variant='latest', market=market,
                                       refresh_days=1)

Dataset "us-shareprices-daily" on disk (0 days old).
- Loading from disk ... Done!
Dataset "us-shareprices-latest" on disk (0 days old).
- Loading from disk ... Done!


In [79]:
df_prices.tail()

SimFinId    Open     Low  ...  Adj. Close  Dividend   Volume
Ticker Date                                  ...                               
low    2019-03-08    186050   99.78   98.00  ...       97.46       NaN  6181344
       2019-03-11    186050   99.83   97.55  ...       98.51       NaN  4598900
       2019-03-12    186050  100.40   99.58  ...       98.96       NaN  4928430
       2019-03-13    186050  101.10  100.61  ...       99.27       NaN  6268161
       2019-03-14    186050  100.98  100.25  ...       98.91       NaN  5393351

[5 rows x 8 columns]

In [0]:
#February 19, 2020: The S&P 500 index reached its highest point in the bull market that started from the low point on March 9, 2009.
start_sp = datetime(2020, 2, 19).strftime('%Y-%m-%d')
end_sp = datetime.today().strftime('%Y-%m-%d')

In [92]:
idx = pd.IndexSlice
df_prices.loc[idx[:,'2019-04-01':'2019-11-30'], :]

,,SimFinId,Open,Low,High,Close,Adj. Close,Dividend,Volume
Ticker,Date,,,,,,,,


In [68]:
df_prices.xs(slice('2007-01-01', '2008-01-01'), level='Date')

,SimFinId,Open,Low,High,Close,Adj. Close,Dividend,Volume
Ticker,,,,,,,,
A,45846,34.99,34.05,35.48,34.30,22.90,NaN,2574600
A,45846,34.30,33.46,34.60,34.41,22.98,NaN,2073700
A,45846,34.30,34.00,34.40,34.09,22.76,NaN,2676600
A,45846,33.98,33.68,34.08,33.97,22.68,NaN,1557200
A,45846,34.08,33.63,34.32,34.01,22.71,NaN,1386200
...,...,...,...,...,...,...,...,...
low,186050,23.19,22.91,23.61,23.47,18.83,NaN,3391300
low,186050,23.42,23.01,23.45,23.19,18.61,NaN,5886300
low,186050,22.94,22.90,23.19,23.16,18.58,NaN,6720000


In [0]:
ticker_symbols_list = ['XLK','MSFT','AAPL','V','MA','INTC','CSCO','ADBE','CRM','NVDA','ACN','AVGO','PYPL','ORCL','IBM','TXN','QCOM','FIS','ADP','INTU','FISV','GPN','AMAT','MU','NOW','ADI','AMD','ADSK','LRCX','CTSH','APH','TEL','HPQ','PAYX','MSI','FLT','KLAC','MCHP','XLNX','GLW','ANSS','HPE','SNPS','VRSN','CDW','KEYS','CDNS','SWKS','MXIM','FTNT','NLOK','NTAP','WDC','IT','AKAM','BR','CTXS','STX','QRVO','LDOS','JKHY','WU','ANET','DXC','JNPR','FFIV','FLIR','XRX','IPGP','ADS']
# create a yearly dataframe with all features, returns (2), nlp, 3 years (2017-2018 on 2019)

In [104]:
close_price_df = pdr.get_data_yahoo(ticker_symbols_list, start_sp, end_sp).reset_index
close_price_df

Attributes        High              ...   Adj Close            
Symbols            XLK        MSFT  ...        IPGP         ADS
Date                                ...                        
2020-02-19  102.940002  188.179993  ...  143.940002  101.360001
2020-02-20  102.820000  187.250000  ...  140.869995  103.059998
2020-02-21  101.290001  183.500000  ...  135.009995  102.160004
2020-02-24   96.709999  174.550003  ...  132.169998   98.720001
2020-02-25   96.540001  174.839996  ...  127.050003   93.750000
2020-02-26   94.709999  173.259995  ...  126.730003   90.940002
2020-02-27   91.559998  167.029999  ...  122.809998   87.699997
2020-02-28   89.209999  163.710007  ...  127.639999   85.879997
2020-03-02   93.500000  172.919998  ...  130.350006   86.070000
2020-03-03   94.660004  175.000000  ...  127.750000   79.680000
2020-03-04   93.820000  170.699997  ...  131.589996   80.690002
2020-03-05   93.029999  170.869995  ...  127.610001   74.209999
2020-03-06   89.440002  163.110001  ...  121.470001   72.610001
2020-03-09   86.000000  157.750000  ...  116.930000   63.020000
2020-03-10   87.650002  161.029999  ...  123.349998   63.560001
2020-03-11   85.820000  157.699997  ...  110.849998   57.279999
2020-03-12   81.900002  153.470001  ...  105.879997   46.820000
2020-03-13   84.379997  161.910004  ...  112.150002   49.509998

[18 rows x 420 columns]

In [107]:
close_price_df.axes

[DatetimeIndex(['2020-02-19', '2020-02-20', '2020-02-21', '2020-02-24',
                '2020-02-25', '2020-02-26', '2020-02-27', '2020-02-28',
                '2020-03-02', '2020-03-03', '2020-03-04', '2020-03-05',
                '2020-03-06', '2020-03-09', '2020-03-10', '2020-03-11',
                '2020-03-12', '2020-03-13'],
               dtype='datetime64[ns]', name='Date', freq=None),
 MultiIndex([(     'High',  'XLK'),
             (     'High', 'MSFT'),
             (     'High', 'AAPL'),
             (     'High',    'V'),
             (     'High',   'MA'),
             (     'High', 'INTC'),
             (     'High', 'CSCO'),
             (     'High', 'ADBE'),
             (     'High',  'CRM'),
             (     'High', 'NVDA'),
             ...
             ('Adj Close', 'JKHY'),
             ('Adj Close',   'WU'),
             ('Adj Close', 'ANET'),
             ('Adj Close',  'DXC'),
             ('Adj Close', 'JNPR'),
             ('Adj Close', 'FFIV'),
          

In [108]:
close_price_df.keys()

MultiIndex([(     'High',  'XLK'),
            (     'High', 'MSFT'),
            (     'High', 'AAPL'),
            (     'High',    'V'),
            (     'High',   'MA'),
            (     'High', 'INTC'),
            (     'High', 'CSCO'),
            (     'High', 'ADBE'),
            (     'High',  'CRM'),
            (     'High', 'NVDA'),
            ...
            ('Adj Close', 'JKHY'),
            ('Adj Close',   'WU'),
            ('Adj Close', 'ANET'),
            ('Adj Close',  'DXC'),
            ('Adj Close', 'JNPR'),
            ('Adj Close', 'FFIV'),
            ('Adj Close', 'FLIR'),
            ('Adj Close',  'XRX'),
            ('Adj Close', 'IPGP'),
            ('Adj Close',  'ADS')],
           names=['Attributes', 'Symbols'], length=420)

In [113]:
close_price_df.reset_index


Attributes        High              ...   Adj Close            
Symbols            XLK        MSFT  ...        IPGP         ADS
Date                                ...                        
2020-02-19  102.940002  188.179993  ...  143.940002  101.360001
2020-02-20  102.820000  187.250000  ...  140.869995  103.059998
2020-02-21  101.290001  183.500000  ...  135.009995  102.160004
2020-02-24   96.709999  174.550003  ...  132.169998   98.720001
2020-02-25   96.540001  174.839996  ...  127.050003   93.750000
2020-02-26   94.709999  173.259995  ...  126.730003   90.940002
2020-02-27   91.559998  167.029999  ...  122.809998   87.699997
2020-02-28   89.209999  163.710007  ...  127.639999   85.879997
2020-03-02   93.500000  172.919998  ...  130.350006   86.070000
2020-03-03   94.660004  175.000000  ...  127.750000   79.680000
2020-03-04   93.820000  170.699997  ...  131.589996   80.690002
2020-03-05   93.029999  170.869995  ...  127.610001   74.209999
2020-03-06   89.440002  163.110001  ...  121.470001   72.610001
2020-03-09   86.000000  157.750000  ...  116.930000   63.020000
2020-03-10   87.650002  161.029999  ...  123.349998   63.560001
2020-03-11   85.820000  157.699997  ...  110.849998   57.279999
2020-03-12   81.900002  153.470001  ...  105.879997   46.820000
2020-03-13   84.379997  161.910004  ...  112.150002   49.509998

[18 rows x 420 columns]

In [0]:
#df = df_unindexed.set_index(['Fruit', 'Color'])
close_price_df.transpose().set_index(['Symbols', 'Date'])
close_price_df